# Task Overview

In this task, your goal is to verify the impact of data noise level in neural network training.
You should use MLP architecture trained on MNIST dataset (like in previous lab exercises).


We will experiment with two setups:
1. Pick X. Take X% of training examples and reassign their labels to random ones. Note that we don't change anything in the test set.
2. Pick X. During each training step, for each sample, change values of X% randomly selected pixels to random values. Note that we don't change anything in the test set.

For both setups, check the impact of various levels of noise (various values of X%) on model performance. Show plots comparing crossentropy (log-loss) and accuracy with varying X%, and also comparing two setups with each other.
Prepare short report briefly explaining the results and observed trends. Consider questions like "why accuracy/loss increases/decreases so quickly/slowly", "why Z is higher in setup 1/2" and any potentially surprising things you see on charts.

### Potential questions, clarifications
* Q: Can I still use sigmoid/MSE loss?
  * You should train your network with softmax and crossentropy loss (log-loss), especially since you should report crossentropy loss.
* Q: When I pick X% of pixels/examples, does it have to be exactly X% or can it be X% in expectation?
  * A: It's fine either way.
* Q: When I randomize pixels, should I randomize them again each time a particular example is drawn (each training step/epoch) or only once before training?
  * A: Each training step/epoch.
* Q: When I randomize labels, should I randomize them again each time a particular example is drawn (each training step/epoch) or only once before training?
  * A: Only once before training.
* Q: What is the expected length of report/explanation?
  * A: There is no minimum/maximum, but between 5 (concise) and 20 sentences should be good. Don't forget about plots.
* Q: When I replace labels/pixels with random values, what random distribution should I use?
  * A: A distribution reasonably similar to the data. However, you don't need to match dataset's distribution exactly - approximation will be totally fine, especially if it's faster or easier to get.
* Q: Can I use something different than Colab/Jupyter Notebook? E.g. just Python files.
  * A: Yes, although notebook is encouraged; please include in you solution code and pdf.

# Model definition and training.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
import random
import math
import numpy as np
import pandas as pd

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        #output = F.softmax(x, dim=1)
        output = x
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        print(loss)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = F.softmax(output, dim=1).argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return (test_loss, 100. * correct / len(test_loader.dataset))
    
batch_size = 256
test_batch_size = 1000
epochs = 10
lr = 1e-2
use_cuda = False
seed = 1
log_interval = 10

use_cuda = not use_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
dataset1[0][1]

5

In [ ]:
batch = 1

for i_batch, sample_batch in enumerate(train_loader):
  if i_batch > 0:
    break
  batch = sample_batch

In [ ]:
batch[1]

tensor([3, 5, 7, 0, 6, 6, 6, 9, 3, 6, 1, 4, 5, 7, 7, 8, 0, 6, 7, 2, 3, 9, 6, 9,
        5, 6, 9, 2, 4, 1, 4, 5, 9, 0, 1, 7, 6, 8, 5, 6, 2, 0, 1, 1, 6, 6, 9, 0,
        7, 1, 5, 9, 3, 3, 4, 0, 9, 0, 6, 3, 7, 9, 2, 4, 6, 7, 0, 8, 9, 2, 3, 0,
        7, 0, 5, 7, 4, 2, 5, 3, 9, 3, 5, 2, 7, 9, 9, 8, 2, 6, 2, 6, 4, 7, 0, 4,
        5, 3, 0, 0, 5, 1, 9, 3, 5, 6, 0, 5, 3, 0, 3, 5, 9, 1, 4, 8, 4, 1, 8, 9,
        3, 1, 0, 5, 7, 6, 1, 1, 5, 7, 4, 0, 3, 1, 2, 9, 2, 5, 4, 1, 9, 1, 7, 9,
        3, 6, 6, 3, 2, 9, 1, 8, 4, 1, 3, 2, 9, 2, 4, 0, 6, 7, 0, 5, 3, 9, 8, 1,
        3, 6, 3, 6, 9, 2, 1, 4, 7, 4, 9, 7, 3, 8, 4, 2, 3, 9, 4, 8, 8, 6, 3, 9,
        1, 1, 6, 9, 0, 7, 7, 9, 9, 5, 8, 9, 0, 1, 7, 3, 7, 8, 0, 6, 1, 5, 2, 7,
        7, 0, 5, 1, 7, 3, 5, 1, 6, 7, 8, 2, 7, 0, 0, 4, 7, 2, 2, 8, 1, 1, 7, 6,
        0, 6, 5, 4, 3, 9, 4, 1, 6, 1, 5, 8, 8, 5, 5, 4])

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f3e53f67b50>

In [ ]:
dataset1

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [ ]:
base_losses = []
base_accs = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    base_losses.append(loss)
    base_accs.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300111
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.802997
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.518391
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.467879
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.248896
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.263700
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.281972
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.270423
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.206838
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.212871
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.222377
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.233204
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.196727
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.191324
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.145970
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.204140
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.222821
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.203892
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.148888
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.276282
Train Epoc

# Training models in setup 1: with randomized labels.

Below I perform training for several values of X% changing labels. Info about train loss is printed directly, plots showing test accuracy/loss are placed in the last part of the notebook.

In [ ]:
class DatasetChangeLabels(Dataset):
  def __init__(self, x, dataset, transform=None):
    self.x = x
    self.dataset = dataset
    self.transform = transform

    length = len(dataset)
    percent = x
    vals = np.random.randint(0, 9, length)
    rand = np.random.uniform(0, 100, length)
    targets = self.dataset.targets
    targets[rand < percent] = torch.tensor(vals[rand < percent])
    self.dataset.targets = targets

  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    sample = self.dataset[idx][0]
    target = self.dataset.targets[idx]

    if self.transform:
      sample = self.transform(sample)

    return (sample, target)

In [ ]:
# Training for x=10
x = 10 # x is value (in per cent) of labels to change
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)

dataset_labels = DatasetChangeLabels(x, dataset1)

train_loader = torch.utils.data.DataLoader(dataset_labels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
labels_losses_10 = []
labels_accs_10 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    labels_losses_10.append(loss)
    labels_accs_10.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304064
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.038069
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.867349
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.813761
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.842053
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.689525
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.818674
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.788170
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.892456
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.886407
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.856125
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.751827
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.773999
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.608325
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.712451
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.578063
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.780483
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.746271
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.724904
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.614673
Train Epoc

In [ ]:
# Training for x=30
x = 30 # x is value (in per cent) of labels to change
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)

dataset_labels = DatasetChangeLabels(x, dataset1)

train_loader = torch.utils.data.DataLoader(dataset_labels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
labels_losses_30 = []
labels_accs_30 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    labels_losses_30.append(loss)
    labels_accs_30.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303994
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.636958
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.461762
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.338216
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.116506
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.435389
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.380920
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.290485
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.400464
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.323424
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.308689
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.381233
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.319702
Train Epoch: 1 [33280/60000 (55%)]	Loss: 1.314726
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.470855
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.390735
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.207134
Train Epoch: 1 [43520/60000 (72%)]	Loss: 1.539594
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.343199
Train Epoch: 1 [48640/60000 (81%)]	Loss: 1.357964
Train Epoc

In [ ]:
# Training for x=70
x = 70 # x is value (in per cent) of labels to change
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)

dataset_labels = DatasetChangeLabels(x, dataset1)

train_loader = torch.utils.data.DataLoader(dataset_labels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
labels_losses_70 = []
labels_accs_70 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    labels_losses_70.append(loss)
    labels_accs_70.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297559
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.151325
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.096946
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.132227
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.141290
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.124593
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.111044
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.028576
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.139420
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.093708
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.069408
Train Epoch: 1 [28160/60000 (47%)]	Loss: 2.109321
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.080352
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.125596
Train Epoch: 1 [35840/60000 (60%)]	Loss: 2.099057
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.088655
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.048531
Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.102247
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.059837
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.073844
Train Epoc

In [ ]:
# Training for x=90
x = 90 # x is value (in per cent) of labels to change
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)

dataset_labels = DatasetChangeLabels(x, dataset1)

train_loader = torch.utils.data.DataLoader(dataset_labels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
labels_losses_90 = []
labels_accs_90 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    labels_losses_90.append(loss)
    labels_accs_90.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308228
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.219749
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.238077
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.202610
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.202631
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.242622
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.214613
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.210890
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.221050
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.213645
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.235723
Train Epoch: 1 [28160/60000 (47%)]	Loss: 2.211447
Train Epoch: 1 [30720/60000 (51%)]	Loss: 2.208185
Train Epoch: 1 [33280/60000 (55%)]	Loss: 2.208039
Train Epoch: 1 [35840/60000 (60%)]	Loss: 2.229577
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.223839
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.209461
Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.200423
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.221804
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.214411
Train Epoc

In [ ]:
# Save results of losses to csv file
label_losses_10 = pd.DataFrame(labels_losses_10, columns=['Loss'])
label_losses_10['Percent'] = 10
label_losses_30 = pd.DataFrame(labels_losses_30, columns=['Loss'])
label_losses_30['Percent'] = 30
label_losses_70 = pd.DataFrame(labels_losses_70, columns=['Loss'])
label_losses_70['Percent'] = 70
label_losses_90 = pd.DataFrame(labels_losses_90, columns=['Loss'])
label_losses_90['Percent'] = 90
label_losses = pd.concat([label_losses_10, label_losses_30, 
                            label_losses_70,
                            label_losses_90])
label_losses.to_csv('./label_losses.csv')

In [ ]:
# Save results of accuracies to csv file
label_accs_10 = pd.DataFrame(labels_accs_10, columns=['Accuracy'])
label_accs_10['Percent'] = 10
label_accs_30 = pd.DataFrame(labels_accs_30, columns=['Accuracy'])
label_accs_30['Percent'] = 30
label_accs_70 = pd.DataFrame(labels_accs_70, columns=['Accuracy'])
label_accs_70['Percent'] = 70
label_accs_90 = pd.DataFrame(labels_accs_90, columns=['Accuracy'])
label_accs_90['Percent'] = 90
label_accs = pd.concat([label_accs_10, label_accs_30, 
                            label_accs_70,
                            label_accs_90])
label_accs.to_csv('./label_accs.csv')

# Training models in setup 2: with randomized pixels.

In [ ]:
# Function to modify X% of pexels
# Approximate distribution (based on histogram of dataset):
# After norming we have prob 0.85 of -0.43 and 0.15 of 2.78
def change_pixels(img, x):
  rng = 28*28
  to_change = math.floor(x*rng/100)
  coords_change = random.sample(range(rng), to_change)
  rand = np.random.uniform(0, 1, to_change)
  for i in range(0, to_change):
    number = coords_change[i]
    x_coord = np.int(math.floor(number/28))
    y_coord = np.int(number%28)
    img[0, x_coord, y_coord] = -0.43 if rand[i] < 0.85 else 2.78 #random.uniform(-0.42, 2.8)
  return img

In [ ]:
class DatasetChangePexels(Dataset):
  def __init__(self, x, dataset, transform=None):
    self.x = x
    self.dataset = dataset
    self.transform = transform

  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    sample = change_pixels(self.dataset[idx][0], self.x)
    target = self.dataset.targets[idx]

    if self.transform:
      sample = self.transform(sample)

    return (sample, target)

Below I perform training for several values of X% changing pexels. Info about train loss is printed directly, plots showing test accuracy/loss are placed in the next part of the notebook.

In [ ]:
# dataset for changing 5 percent of pexels
dataset1_change_pexels = DatasetChangePexels(5, dataset1)
train_loader = torch.utils.data.DataLoader(dataset1_change_pexels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
test_losses_5 = []
test_accuracies_5 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    test_losses_5.append(loss)
    test_accuracies_5.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305732
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.574649
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.399738
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.314333
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.296659
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.322055
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.188276
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.250051
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.208022
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.253478
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.160699
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.171953
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.249773
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.265347
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.187048
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.209895
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.355840
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.197884
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.116470
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.105023
Train Epoc

In [ ]:
# dataset for changing 10 percent of pexels
dataset1_change_pexels = DatasetChangePexels(10, dataset1)
train_loader = torch.utils.data.DataLoader(dataset1_change_pexels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
test_losses_10 = []
test_accuracies_10 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    test_losses_10.append(loss)
    test_accuracies_10.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303460
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.631058
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.398602
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.395699
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.328922
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.343915
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.243829
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.249543
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.165278
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.335876
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.154979
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.163842
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.158318
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.236014
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.198482
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.202552
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.262791
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.126436
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.151501
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.169679
Train Epoc

In [ ]:
# dataset for changing 15 percent of pexels
dataset1_change_pexels = DatasetChangePexels(15, dataset1)
train_loader = torch.utils.data.DataLoader(dataset1_change_pexels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
test_losses_15 = []
test_accuracies_15 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    test_losses_15.append(loss)
    test_accuracies_15.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298139
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.711016
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.398009
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.346000
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.276330
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.245866
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.344094
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.325255
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.288058
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.243402
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.273582
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.195106
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.190198
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.321431
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.312064
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.164598
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.250449
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.178670
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.197737
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.206695
Train Epoc

In [ ]:
# dataset for changing 20 percent of pexels
dataset1_change_pexels = DatasetChangePexels(20, dataset1)
train_loader = torch.utils.data.DataLoader(dataset1_change_pexels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
test_losses_20 = []
test_accuracies_20 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    test_losses_20.append(loss)
    test_accuracies_20.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307561
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.683067
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.480514
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.357844
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.348191
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.304313
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.284673
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.260531
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.264934
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.221315
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.190313
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.210679
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.158159
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.205014
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.196204
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.203895
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.205893
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.195507
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.222456
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.189693
Train Epoc

In [ ]:
# dataset for changing 30 percent of pexels
dataset1_change_pexels = DatasetChangePexels(30, dataset1)
train_loader = torch.utils.data.DataLoader(dataset1_change_pexels,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
test_losses_30 = []
test_accuracies_30 = []

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    loss, acc = test(model, device, test_loader)
    test_losses_30.append(loss)
    test_accuracies_30.append(acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299093
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.675417
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.541822
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.417877
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.326981
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.351227
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.443661
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.279979
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.331923
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.320142
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.379576
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.243086
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.346520
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.258000
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.356737
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.258223
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.275393
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.340387
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.375464
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.283287
Train Epoc

In [ ]:
# Save results of losses to csv file
pexel_losses_5 = pd.DataFrame(test_losses_5, columns=['Loss'])
pexel_losses_5['Percent'] = 5
pexel_losses_10 = pd.DataFrame(test_losses_10, columns=['Loss'])
pexel_losses_10['Percent'] = 10
pexel_losses_15 = pd.DataFrame(test_losses_15, columns=['Loss'])
pexel_losses_15['Percent'] = 15
pexel_losses_20 = pd.DataFrame(test_losses_20, columns=['Loss'])
pexel_losses_20['Percent'] = 20
pexel_losses_30 = pd.DataFrame(test_losses_30, columns=['Loss'])
pexel_losses_30['Percent'] = 30
pexel_losses = pd.concat([pexel_losses_5, pexel_losses_10, 
                            pexel_losses_15, pexel_losses_20,
                            pexel_losses_30])
pexel_losses.to_csv('./pexel_losses.csv')

In [ ]:
# Save results of accuracies to csv file
pexel_accs_5 = pd.DataFrame(test_accuracies_5, columns=['Accuracy'])
pexel_accs_5['Percent'] = 5
pexel_accs_10 = pd.DataFrame(test_accuracies_10, columns=['Accuracy'])
pexel_accs_10['Percent'] = 10
pexel_accs_15 = pd.DataFrame(test_accuracies_15, columns=['Accuracy'])
pexel_accs_15['Percent'] = 15
pexel_accs_20 = pd.DataFrame(test_accuracies_20, columns=['Accuracy'])
pexel_accs_20['Percent'] = 20
pexel_accs_30 = pd.DataFrame(test_accuracies_30, columns=['Accuracy'])
pexel_accs_30['Percent'] = 30
pexel_accs = pd.concat([pexel_accs_5, pexel_accs_10, 
                            pexel_accs_15, pexel_accs_20,
                            pexel_accs_30])
pexel_accs.to_csv('./pexel_accs.csv')

# Plots and report.

In [ ]:
base_losses_frame = pd.DataFrame(base_losses, columns=['Loss'])
base_losses_frame['Percent'] = 0
base_accs_frame = pd.DataFrame(base_accs, columns=['Accuracy'])
base_accs_frame['Percent'] = 0

In [ ]:
# Test losses for changing labels
label_losses = pd.read_csv('./label_losses.csv')
label_losses = pd.concat([label_losses, base_losses_frame])
px.line(label_losses, y='Loss', color='Percent', title="Test losses for changing labels")

In [ ]:
# Test accuracies for changing labels
label_accs = pd.read_csv('./label_accs.csv')
label_accs = pd.concat([label_accs, base_accs_frame])
px.line(label_accs, y='Accuracy', color='Percent', title="Test accuracies for changing labels")

In [ ]:
# Test accuracies for changing labels (without 90%)
label_accs = pd.read_csv('./label_accs.csv')
label_accs = pd.concat([label_accs, base_accs_frame])
label_accs = pd.DataFrame(label_accs[label_accs['Percent'] != 90])
px.line(label_accs, y='Accuracy', color='Percent', title="Test accuracies for changing labels (without 90%)")

In [ ]:
# Test losses for changing pexels
pexel_losses = pd.read_csv('./pexel_losses.csv')
pexel_losses = pd.concat([pexel_losses, base_losses_frame])
px.line(pexel_losses, y='Loss', color='Percent', title="Test losses for changing pexels")

In [ ]:
# Test accuracies for changing pexels
pexel_accs = pd.read_csv('./pexel_accs.csv')
pexel_accs = pd.concat([pexel_accs, base_accs_frame])
px.line(pexel_accs, y='Accuracy', color='Percent', title="Test accuracies for changing pexels")

In [ ]:
# Comparison of loss between both approaches
label_10 = pd.DataFrame(label_losses[label_losses['Percent'] == 10])
label_10['Experiment'] = '10% (labels)'
label_30 = pd.DataFrame(label_losses[label_losses['Percent'] == 30])
label_30['Experiment'] = '30% (labels)'
pexel_15 = pd.DataFrame(pexel_losses[pexel_losses['Percent'] == 15])
pexel_15['Experiment'] = '15% (pexels)'
pexel_20 = pd.DataFrame(pexel_losses[pexel_losses['Percent'] == 20])
pexel_20['Experiment'] = '20% (pexels)'
base_0 = base_losses_frame
base_0['Experiment'] = 'No change'
plot1_frame = pd.concat([label_10, label_30, pexel_15, pexel_20, base_0])

px.line(plot1_frame, y='Loss', color='Experiment', title='Comparison of loss for some values for two approaches')

In [ ]:
# Comparison of accuracy between both approaches
label_10 = pd.DataFrame(label_accs[label_accs['Percent'] == 10])
label_10['Experiment'] = '10% (labels)'
label_30 = pd.DataFrame(label_accs[label_accs['Percent'] == 30])
label_30['Experiment'] = '30% (labels)'
pexel_15 = pd.DataFrame(pexel_accs[pexel_accs['Percent'] == 15])
pexel_15['Experiment'] = '15% (pexels)'
pexel_20 = pd.DataFrame(pexel_accs[pexel_accs['Percent'] == 20])
pexel_20['Experiment'] = '20% (pexels)'
base_0 = base_accs_frame
base_0['Experiment'] = 'No change'
plot1_frame = pd.concat([label_10, label_30, pexel_15, pexel_20, base_accs_frame])

px.line(plot1_frame, y='Accuracy', color='Experiment', title='Comparison of accuracy for some values for two approaches')

In the experiment I examined two approaches to adding noise to data. In the first setting, I analysed results of changing: 10%, 30%, 70% and 90% of labels to random ones. We can tell for sure that changing 90% of labels is not a particularly good idea: it makes the results significantly worse. All of the other values give comparable results. From plots we can see that the best effect gives changing 10% of labels. However, over the first ten training epochs, in this setting I was not able to achieve better results of training than without making any change.

Interestingly, even if we change 70 per cent of labels to random ones, the network is able to achive accuracy of more than 90%. However, changing labels makes the training process slower (we need more epochs to achieve the same accuracy this way). This is because the network is regularly pushed in random directions, which takes time, but does not help to learn.

In the second approach, I analysed changing: 5%, 10%, 15%, 20% and 30% of pexels to random values, according to approximate random distribution of values of pexels in MNIST images. Changing 5% of pexels seems to give the worst results, which are also similar to the situation with no change at all. This is reasonable, as most of the pexels are white and also if we pick a pixel, most likely it will be set as white according to the distribution. In effect, randomly changing 5% of pexels does not usually result in a lot of changes. Values for changing 10%, 15%, 20% and 30% are comparable. In general, adding noise in second approach seems to make the training process faster and more stable. From epoch 6/7 we can see effects of overfit in training without any noise: the loss is getting bigger and the accuracy smaller. If we randomly change pexels, this problem can not be seen on plots.

Generally we can see that the accuracies/losses for the second setting are better than without making any change. The reason is probably that adding some noise makes it less likely that the model will overfit training examples, therefore it generalises better.

On the last two graphs we can compare values from setting 1 and 2. We can see that the first approach makes the situation worse or similar to making no change. However, the second approach helps to achive better results.